# LAB 4a:  Creating a Sampled Dataset.

**Learning Objectives**

1. Setup up the environment
1. Sample the natality dataset to create train/eval/test sets
1. Preprocess the data in Pandas dataframe


## Introduction 
In this notebook, we'll read data from BigQuery into our notebook to preprocess the data within a Pandas dataframe for a small, repeatable sample.

We will set up the environment, sample the natality dataset to create train/eval/test splits, and preprocess the data in a Pandas dataframe.

## Set up environment variables and load necessary libraries

Check that the Google BigQuery library is installed and if not, install it. 

In [1]:
%%bash
sudo pip freeze | grep google-cloud-bigquery==1.6.1 || \
sudo pip install google-cloud-bigquery==1.6.1

google-cloud-bigquery==1.6.1


Import necessary libraries.

In [2]:
from google.cloud import bigquery
import pandas as pd

Set environment variables so that we can use them throughout the entire lab. We will be using our project name for our bucket, so you only need to change your project and region.

In [3]:
%%bash
export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-04-8722038efd75


In [4]:
#PROJECT = "cloud-training-demos"  # Replace with your PROJECT
PROJECT = "qwiklabs-gcp-04-8722038efd75"

## Create ML datasets by sampling using BigQuery

We'll begin by sampling the BigQuery data to create smaller datasets. Let's create a BigQuery client that we'll use throughout the lab.

In [5]:
bq = bigquery.Client(project = PROJECT)

We need to figure out the right way to divide our hash values to get our desired splits. To do that we need to define some values to hash with in the modulo. Feel free to play around with these values to get the perfect combination.

In [6]:
modulo_divisor = 100
train_percent = 80.0
eval_percent = 10.0

train_buckets = int(modulo_divisor * train_percent / 100.0)
eval_buckets = int(modulo_divisor * eval_percent / 100.0)

We can make a series of queries to check if our bucketing values result in the correct sizes of each of our dataset splits and then adjust accordingly. Therefore, to make our code more compact and reusable, let's define a function to return the head of a dataframe produced from our queries up to a certain number of rows.

In [7]:
def display_dataframe_head_from_query(query, count=10):
    """Displays count rows from dataframe head from query.
    
    Args:
        query: str, query to be run on BigQuery, results stored in dataframe.
        count: int, number of results from head of dataframe to display.
    Returns:
        Dataframe head with count number of results.
    """
    df = bq.query(
        query + " LIMIT {limit}".format(
            limit=count)).to_dataframe()

    return df.head(count)

For our first query, we're going to use the original query above to get our label, features, and columns to combine into our hash which we will use to perform our repeatable splitting. There are only a limited number of years, months, days, and states in the dataset. Let's see what the hash values are. We will need to include all of these extra columns to hash on to get a fairly uniform spread of the data. Feel free to try less or more in the hash and see how it changes your results.

In [8]:
# Get label, features, and columns to hash and split into buckets
hash_cols_fixed_query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
        WHEN day IS NULL THEN
            CASE
                WHEN wday IS NULL THEN 0
                ELSE wday
            END
        ELSE day
    END AS date,
    IFNULL(state, "Unknown") AS state,
    IFNULL(mother_birth_state, "Unknown") AS mother_birth_state
FROM
    publicdata.samples.natality
WHERE
    year > 2000
    AND weight_pounds > 0
    AND mother_age > 0
    AND plurality > 0
    AND gestation_weeks > 0
"""

display_dataframe_head_from_query(hash_cols_fixed_query)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,year,month,date,state,mother_birth_state
0,7.063611,True,32,1,37,2001,12,3,CO,CA
1,4.687028,True,30,3,33,2001,6,5,IN,IN
2,7.561856,True,20,1,39,2001,4,5,MN,MN
3,7.561856,True,31,1,37,2001,10,5,MS,MS
4,7.312733,True,32,1,40,2001,11,3,MO,MO
5,7.627994,False,30,1,40,2001,10,5,NY,PA
6,7.251004,True,33,1,37,2001,11,5,WA,WA
7,7.500126,False,23,1,39,2001,9,2,OK,LA
8,7.125340,False,33,1,39,2001,1,4,TX,MS
9,7.749249,True,31,1,39,2001,1,1,TX,Foreign


Using `COALESCE` would provide the same result as the nested `CASE WHEN`. This is preferable when all we want is the first non-null instance. To be precise the `CASE WHEN` would become `COALESCE(wday, day, 0) AS date`. You can read more about it [here](https://cloud.google.com/bigquery/docs/reference/standard-sql/conditional_expressions).

Next query will combine our hash columns and will leave us just with our label, features, and our hash values.

In [9]:
data_query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING),
            CAST(date AS STRING),
            CAST(state AS STRING),
            CAST(mother_birth_state AS STRING)
        )
    ) AS hash_values
FROM
    ({CTE_hash_cols_fixed})
""".format(CTE_hash_cols_fixed=hash_cols_fixed_query)

display_dataframe_head_from_query(data_query)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,7.063611,True,32,1,37,4762325092919148672
1,4.687028,True,30,3,33,2341060194216507348
2,7.561856,True,20,1,39,-8842767231851202242
3,7.561856,True,31,1,37,7957807816914159435
4,7.312733,True,32,1,40,-5961624242430066305
5,7.627994,False,30,1,40,5493295634082918412
6,7.251004,True,33,1,37,-2988893757655690534
7,7.500126,False,23,1,39,-6735199252008114417
8,7.125340,False,33,1,39,-3514093303120687641
9,7.749249,True,31,1,39,2175328516857391398


The next query is going to find the counts of each of the unique 657484 `hash_values`. This will be our first step at making actual hash buckets for our split via the `GROUP BY`.

In [10]:
# Get the counts of each of the unique hashs of our splitting column
first_bucketing_query = """
SELECT
    hash_values,
    COUNT(*) AS num_records
FROM
    ({CTE_data})
GROUP BY
    hash_values
""".format(CTE_data=data_query)

display_dataframe_head_from_query(first_bucketing_query)

,hash_values,num_records
0,2341060194216507348,696
1,-8842767231851202242,515
2,7957807816914159435,369
3,-746421027886559730,167
4,-7566476151165360246,87
5,315818780995586851,400
6,8711610669332498583,7
7,-4170581330234584329,30
8,-4741381250325891292,1357
9,19168170974013054,503


The query below performs a second layer of bucketing where now for each of these bucket indices we count the number of records.

In [11]:
# Get the number of records in each of the hash buckets
second_bucketing_query = """
SELECT
    ABS(MOD(hash_values, {modulo_divisor})) AS bucket_index,
    SUM(num_records) AS num_records
FROM
    ({CTE_first_bucketing})
GROUP BY
    ABS(MOD(hash_values, {modulo_divisor}))
""".format(
    CTE_first_bucketing=first_bucketing_query, modulo_divisor=modulo_divisor)

display_dataframe_head_from_query(second_bucketing_query)

,bucket_index,num_records
0,5,449280
1,34,379000
2,7,270933
3,40,333712
4,19,384793
5,95,313544
6,81,233538
7,24,352559
8,20,432535
9,43,201054


The number of records is hard for us to easily understand the split, so we will normalize the count into percentage of the data in each of the hash buckets in the next query.

In [12]:
# Calculate the overall percentages
percentages_query = """
SELECT
    bucket_index,
    num_records,
    CAST(num_records AS FLOAT64) / (
    SELECT
        SUM(num_records)
    FROM
        ({CTE_second_bucketing})) AS percent_records
FROM
    ({CTE_second_bucketing})
""".format(CTE_second_bucketing=second_bucketing_query)

display_dataframe_head_from_query(percentages_query)

,bucket_index,num_records,percent_records
0,17,222562,0.006742
1,46,281627,0.008531
2,6,548778,0.016624
3,96,529357,0.016036
4,48,370308,0.011218
5,14,251675,0.007624
6,29,453175,0.013728
7,92,336735,0.010201
8,97,480790,0.014564
9,25,158301,0.004795


We'll now select the range of buckets to be used in training.

In [13]:
# Choose hash buckets for training and pull in their statistics
train_query = """
SELECT
    *,
    "train" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= 0
    AND bucket_index < {train_buckets}
""".format(
    CTE_percentages=percentages_query,
    train_buckets=train_buckets)

display_dataframe_head_from_query(train_query)

,bucket_index,num_records,percent_records,dataset_name
0,17,222562,0.006742,train
1,46,281627,0.008531,train
2,6,548778,0.016624,train
3,48,370308,0.011218,train
4,14,251675,0.007624,train
5,29,453175,0.013728,train
6,25,158301,0.004795,train
7,12,412875,0.012507,train
8,53,230298,0.006976,train
9,49,468121,0.014181,train


We'll do the same by selecting the range of buckets to be used evaluation.

In [14]:
# Choose hash buckets for validation and pull in their statistics
eval_query = """
SELECT
    *,
    "eval" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= {train_buckets}
    AND bucket_index < {cum_eval_buckets}
""".format(
    CTE_percentages=percentages_query,
    train_buckets=train_buckets,
    cum_eval_buckets=train_buckets + eval_buckets)

display_dataframe_head_from_query(eval_query)

,bucket_index,num_records,percent_records,dataset_name
0,83,411258,0.012458,eval
1,80,312489,0.009466,eval
2,86,274489,0.008315,eval
3,88,423809,0.012838,eval
4,81,233538,0.007074,eval
5,87,523881,0.015870,eval
6,82,468179,0.014182,eval
7,84,341155,0.010334,eval
8,89,256482,0.007770,eval
9,85,368045,0.011149,eval


Lastly, we'll select the hash buckets to be used for the test split.

In [15]:
# Choose hash buckets for testing and pull in their statistics
test_query = """
SELECT
    *,
    "test" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= {cum_eval_buckets}
    AND bucket_index < {modulo_divisor}
""".format(
    CTE_percentages=percentages_query,
    cum_eval_buckets=train_buckets + eval_buckets,
    modulo_divisor=modulo_divisor)

display_dataframe_head_from_query(test_query)

,bucket_index,num_records,percent_records,dataset_name
0,98,374697,0.011351,test
1,91,333267,0.010096,test
2,96,529357,0.016036,test
3,90,286465,0.008678,test
4,97,480790,0.014564,test
5,93,215710,0.006534,test
6,99,223334,0.006765,test
7,95,313544,0.009498,test
8,92,336735,0.010201,test
9,94,431001,0.013056,test


In the below query, we'll `UNION ALL` all of the datasets together so that all three sets of hash buckets will be within one table. We added `dataset_id` so that we can sort on it in the query after.

In [16]:
# Union the training, validation, and testing dataset statistics
union_query = """
SELECT
    0 AS dataset_id,
    *
FROM
    ({CTE_train})
UNION ALL
SELECT
    1 AS dataset_id,
    *
FROM
    ({CTE_eval})
UNION ALL
SELECT
    2 AS dataset_id,
    *
FROM
    ({CTE_test})
""".format(CTE_train=train_query, CTE_eval=eval_query, CTE_test=test_query)

display_dataframe_head_from_query(union_query)

,dataset_id,bucket_index,num_records,percent_records,dataset_name
0,2,96,529357,0.016036,test
1,2,92,336735,0.010201,test
2,2,97,480790,0.014564,test
3,2,90,286465,0.008678,test
4,2,91,333267,0.010096,test
5,2,98,374697,0.011351,test
6,2,94,431001,0.013056,test
7,2,99,223334,0.006765,test
8,2,95,313544,0.009498,test
9,2,93,215710,0.006534,test


Lastly, we'll show the final split between train, eval, and test sets. We can see both the number of records and percent of the total data. It is really close to the 80/10/10 that we were hoping to get.

In [18]:
# Show final splitting and associated statistics
split_query = """
SELECT
    dataset_id,
    dataset_name,
    SUM(num_records) AS num_records,
    SUM(percent_records) AS percent_records
FROM
    ({CTE_union})
GROUP BY
    dataset_id,
    dataset_name
ORDER BY
    dataset_id
""".format(CTE_union=union_query)

display_dataframe_head_from_query(split_query)

,dataset_id,dataset_name,num_records,percent_records
0,0,train,25873134,0.783765
1,1,eval,3613325,0.109457
2,2,test,3524900,0.106778


Now that we know that our splitting values produce a good global splitting on our data, here's a way to get a well-distributed portion of the data in such a way that the train/eval/test sets do not overlap and takes a subsample of our global splits.

In [19]:
# every_n allows us to subsample from each of the hash values
# This helps us get approximately the record counts we want
every_n = 1000

splitting_string = "ABS(MOD(hash_values, {0} * {1}))".format(every_n, modulo_divisor)

def create_data_split_sample_df(query_string, splitting_string, lo, up):
    """Creates a dataframe with a sample of a data split.

    Args:
        query_string: str, query to run to generate splits.
        splitting_string: str, modulo string to split by.
        lo: float, lower bound for bucket filtering for split.
        up: float, upper bound for bucket filtering for split.
    Returns:
        Dataframe containing data split sample.
    """
    query = "SELECT * FROM ({0}) WHERE {1} >= {2} and {1} < {3}".format(
        query_string, splitting_string, int(lo), int(up))

    df = bq.query(query).to_dataframe()

    return df

train_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=0, up=train_percent)

eval_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=train_percent, up=train_percent + eval_percent)

test_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=train_percent + eval_percent, up=modulo_divisor)

print("There are {} examples in the train dataset.".format(len(train_df)))
print("There are {} examples in the validation dataset.".format(len(eval_df)))
print("There are {} examples in the test dataset.".format(len(test_df)))

There are 7733 examples in the train dataset.
There are 1037 examples in the validation dataset.
There are 561 examples in the test dataset.


## Preprocess data using Pandas

We'll perform a few preprocessing steps to the data in our dataset. Let's add extra rows to simulate the lack of ultrasound. That is we'll duplicate some rows and make the `is_male` field be `Unknown`. Also, if there is more than child we'll change the `plurality` to `Multiple(2+)`. While we're at it, we'll also change the plurality column to be a string. We'll perform these operations below. 

Let's start by examining the training dataset as is.

In [20]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,5.937049,True,23,1,39,-6784884401981100070
1,7.374463,True,28,1,38,-1792211462198300014
2,7.810978,True,22,1,42,-2473233337125700055
3,8.999270,True,21,1,38,6567453535821100019
4,6.250105,True,19,1,37,4896699230184800022


Also, notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [21]:
train_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hash_values
count,7733.000000,7733.000000,7733.000000,7733.000000,7.733000e+03
mean,7.264415,28.213371,1.035691,38.691064,-2.984870e+17
std,1.303220,6.134232,0.201568,2.531921,5.590715e+18
min,0.562179,13.000000,1.000000,18.000000,-9.210618e+18
25%,6.624891,23.000000,1.000000,38.000000,-6.781866e+18
50%,7.345803,28.000000,1.000000,39.000000,5.057323e+17
75%,8.062305,33.000000,1.000000,40.000000,4.896699e+18
max,11.563246,48.000000,4.000000,47.000000,9.203641e+18


It is always crucial to clean raw data before using in machine learning, so we have a preprocessing step. We'll define a `preprocess` function below. Note that the mother's age is an input to our model so users will have to provide the mother's age; otherwise, our service won't work. The features we use for our model were chosen because they are such good predictors and because they are easy enough to collect.

In [22]:
def preprocess(df):
    """ Preprocess pandas dataframe for augmented babyweight data.
    
    Args:
        df: Dataframe containing raw babyweight data.
    Returns:
        Pandas dataframe containing preprocessed raw babyweight data as well
            as simulated no ultrasound data masking some of the original data.
    """
    # Clean up raw data
    # Filter out what we don"t want to use for training
    df = df[df.weight_pounds > 0]
    df = df[df.mother_age > 0]
    df = df[df.gestation_weeks > 0]
    df = df[df.plurality > 0]

    # Modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ["Single(1)",
                    "Twins(2)",
                    "Triplets(3)",
                    "Quadruplets(4)",
                    "Quintuplets(5)"]))
    df["plurality"].replace(twins_etc, inplace=True)

    # Clone data and mask certain columns to simulate lack of ultrasound
    no_ultrasound = df.copy(deep=True)

    # Modify is_male
    no_ultrasound["is_male"] = "Unknown"
    
    # Modify plurality
    condition = no_ultrasound["plurality"] != "Single(1)"
    no_ultrasound.loc[condition, "plurality"] = "Multiple(2+)"

    # Concatenate both datasets together and shuffle
    return pd.concat(
        [df, no_ultrasound]).sample(frac=1).reset_index(drop=True)

Let's process the train/eval/test set and see a small sample of the training data after our preprocessing:

In [23]:
train_df = preprocess(train_df)
eval_df = preprocess(eval_df)
test_df = preprocess(test_df)

In [24]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,6.188376,Unknown,24,Single(1),38,8720767384765100051
1,9.393897,Unknown,25,Single(1),38,2749458901984200063
2,7.813183,Unknown,32,Single(1),41,3835474516102300019
3,6.750554,True,21,Single(1),36,6433000297422000014
4,8.126239,True,34,Single(1),40,-3356992025038600017


In [25]:
train_df.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
15461,6.790238,Unknown,31,Single(1),38,780565305641800050
15462,9.232960,False,26,Single(1),38,-6784884401981100070
15463,6.937947,Unknown,24,Single(1),40,-7810040696678700062
15464,6.680007,Unknown,26,Multiple(2+),35,-6784884401981100070
15465,7.874912,Unknown,34,Single(1),40,-6471666456918200072


Let's look again at a summary of the dataset. Note that we only see numeric columns, so `plurality` does not show up.

In [26]:
train_df.describe()

,weight_pounds,mother_age,gestation_weeks,hash_values
count,15466.000000,15466.000000,15466.000000,1.546600e+04
mean,7.264415,28.213371,38.691064,-2.984870e+17
std,1.303178,6.134034,2.531839,5.590534e+18
min,0.562179,13.000000,18.000000,-9.210618e+18
25%,6.624891,23.000000,38.000000,-6.781866e+18
50%,7.345803,28.000000,39.000000,5.057323e+17
75%,8.062305,33.000000,40.000000,4.896699e+18
max,11.563246,48.000000,47.000000,9.203641e+18


## Write to .csv files 

In the final versions, we want to read from files, not Pandas dataframes. So, we write the Pandas dataframes out as csv files. Using csv files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.

In [28]:
# Define columns
columns = ["weight_pounds",
           "is_male",
           "mother_age",
           "plurality",
           "gestation_weeks"]

# Write out CSV files
train_df.to_csv(
    path_or_buf="train.csv", columns=columns, header=False, index=False)
eval_df.to_csv(
    path_or_buf="eval.csv", columns=columns, header=False, index=False)
test_df.to_csv(
    path_or_buf="test.csv", columns=columns, header=False, index=False)

In [29]:
%%bash
wc -l *.csv

  2074 eval.csv
  1122 test.csv
 15466 train.csv
 18662 total


In [30]:
%%bash
head *.csv

==> eval.csv <==
6.3118345610599995,Unknown,35,Single(1),38
5.43659938092,Unknown,21,Multiple(2+),35
7.43839671988,Unknown,20,Single(1),40
6.37576861704,Unknown,27,Multiple(2+),34
7.62358501996,True,30,Single(1),38
7.12534030784,Unknown,22,Single(1),38
6.9996768185,False,32,Single(1),39
4.9714240081,False,26,Single(1),36
5.74083730248,Unknown,35,Single(1),36
8.437090766739999,Unknown,40,Single(1),40

==> test.csv <==
6.9996768185,Unknown,20,Single(1),39
6.9996768185,Unknown,26,Single(1),37
7.93443680938,Unknown,25,Single(1),39
5.5005334369,Unknown,35,Single(1),34
7.31052860792,Unknown,26,Single(1),39
9.12493302418,Unknown,23,Single(1),41
7.68751907594,Unknown,31,Single(1),38
6.37576861704,Unknown,35,Single(1),39
5.8753192823,Unknown,27,Single(1),41
7.936641432,True,22,Single(1),40

==> train.csv <==
6.1883756943399995,Unknown,24,Single(1),38
9.39389698382,Unknown,25,Single(1),38
7.81318256528,Unknown,32,Single(1),41
6.75055446244,True,21,Single(1),36
8.12623897732,True,34,Single(1),40


In [31]:
%%bash
tail *.csv

==> eval.csv <==
5.24920645822,Unknown,38,Single(1),37
5.1367707046,False,42,Single(1),39
7.7492485093,Unknown,21,Single(1),41
8.13285284518,Unknown,20,Single(1),42
6.93794738514,Unknown,18,Single(1),40
7.62578964258,Unknown,20,Single(1),40
7.99837086536,Unknown,34,Single(1),40
6.0627122049999995,Unknown,38,Single(1),42
6.93794738514,True,27,Single(1),40
7.5618555866,True,33,Single(1),39

==> test.csv <==
7.7492485093,True,23,Single(1),40
6.44631654088,True,31,Single(1),39
8.437090766739999,Unknown,22,Single(1),40
9.12493302418,Unknown,31,Single(1),39
7.936641432,Unknown,36,Single(1),40
7.62578964258,False,16,Single(1),39
6.686620406459999,Unknown,24,Single(1),39
8.50102482272,True,22,Single(1),40
9.49971886958,Unknown,29,Single(1),40
7.50012615324,Unknown,30,Single(1),34

==> train.csv <==
7.09227096854,False,31,Single(1),42
7.43839671988,True,21,Single(1),40
7.8991628474599995,True,29,Single(1),40
5.35282372136,False,23,Twins(2),34
7.81318256528,Unknown,21,Single(1),39
6.7902376696,U

## Lab Summary: 
In this lab, we set up the environment, sampled the natality dataset to create train/eval/test splits, and preprocessed the data in a Pandas dataframe.

Copyright 2020 Google LLC
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.